# Creating and deploying an R web service to Azure Kubernetes Service (AKS) with Azure Machine Learning Services
This notebook shows the steps required for writing and deploying an Azure Machine Learning Service which runs R code in AKS.

# Get workspace
Load existing workspace from the config file info.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Register the model
Register an existing trained model, add description and tags.

In [ ]:
from azureml.core.model import Model
model = Model.register(model_path = "model.RData",
                       model_name = "model.RData",
                       tags = {'area': "samples", 'type': "regression"},
                       description = "A simple linear regression model to show the usage of R in Azure Machine Learning Services.",
                       workspace = ws)

print(model.name, model.description, model.version)

# Create an image
Create an image using the registered model the script that will load and run the model.

In [ ]:
%%writefile score.py
from azureml.core.model import Model
import rpy2.rinterface
import rpy2.robjects as robjects

def init():
    # init rpy2
    rpy2.rinterface.initr()
    # load model
    model_path = Model.get_model_path('model.RData')
    robjects.r("load('{model_path}')".format(model_path=model_path))

def run(input_json_string):
    try:
        result_vector = robjects.r(
                "run('{input_json_string}')".format(input_json_string=input_json_string)
            )
        if len(result_vector) > 0:
            return result_vector[0]
        else:
            return ""
    except Exception as e:
        error = str(e)
        return error

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

conda_dependencies = CondaDependencies.create(conda_packages=['rpy2', 'mro-base','r-jsonlite','r-essentials'])

with open("conda_dependencies.yml","w") as f:
    f.write(conda_dependencies.serialize_to_string())

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "conda_dependencies.yml",
                                                  description = "Sample image to show the usage of R with Azure Machine Learning Services.",
                                                  tags = {'area': "samples", 'type': "regression"},
                                                 )

image = ContainerImage.create(name = "r-on-amls-sample",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

# Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [ ]:
aks_name = 'akscluster' 

from azureml.core.compute import AksCompute, ComputeTarget
prov_config = AksCompute.provisioning_configuration()
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [ ]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

## Optional step: Attach existing AKS cluster

If you have existing AKS cluster in your Azure subscription, you can attach it to the Workspace.

In [ ]:
'''
# Use the default configuration (can also provide parameters to customize)
resource_id = '/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourcegroups/raymondsdk0604/providers/Microsoft.ContainerService/managedClusters/my-aks-0605d37425356b7d01'

create_name='my-existing-aks' 
# Create the cluster
attach_config = AksCompute.attach_configuration(resource_id=resource_id)
aks_target = ComputeTarget.attach(workspace=ws, name=create_name, attach_configuration=attach_config)
# Wait for the operation to complete
aks_target.wait_for_completion(True)
'''

# Deploy web service to AKS

In [ ]:
#Set the web service configuration (using default here)
from azureml.core.webservice import AksWebservice
aks_config = AksWebservice.deploy_configuration()

In [ ]:
%%time
from azureml.core.webservice import Webservice
aks_service_name ='r-on-aml-service'
aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

In [ ]:
aks_service.scoring_uri

In [ ]:
aks_service.swagger_uri

In [ ]:
aks_service.get_keys()

Note: By default, the deployed service uses HTTP only. To secure your service see and follow the steps described [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-secure-web-service).

# Test the web service
We test the web sevice by passing data.

In [ ]:
%%time
import json

test_sample = json.dumps({'x': 987.654})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aks_service.run(input_data = test_sample)

print(prediction)

# Clean up
Delete the service, image and model.

In [ ]:
%%time
aks_service.delete()
image.delete()
model.delete()